In [60]:
%%html 
<h1>Original Code from Saraj Ai </h1>
<a>https://www.youtube.com/watch?v=FTr3n7uBIuE&index=8&list=PL2-dafEMk2A7mu0bSksCGMJEmeddU_H4D</a>
<h2>github:<a><h5>https://www.youtube.com/redirect?v=FTr3n7uBIuE&event=video_description&redir_token=Vh1Z2mNefWEXxy5oFF9FJak7A9h8MTU0OTAyODQyN0AxNTQ4OTQyMDI3&q=https%3A%2F%2Fgithub.com%2FllSourcell%2FConvolutional_neural_network</h5></a></h2>

In [54]:
import numpy as np

In [55]:
%%html
<h3>
    Process CNN 
    <h5>
        <ul>
            <li>Prepare dataset</li>
            <li>Convolution</li>
            <li>Relu activation</li>
            <li>Pooling</li>
            <li>Dropout</li>
            <li>Flatten</li>
            <li>Dense</li>
            <li>Softmax</li>
        </ul>
    </h5>

</h3>

In [62]:
class CNN:
    def __init__(self):
        self.layer = []
        self.pool_size = None
        
    def load_weights(self, weights):
        assert not self.layers, "Weights can only be loaded once!"
        #add the saved matrix values to the convolutional network
        for k in range(len(weights.keys())):
            self.layers.append(weights['layer_{}'.format(k)])
            
    def predict(self,X):
        h = self.cnn_layer(X, layer_i=0,border_mode="full");X=h #*difference
        h = self.relu_layer(X);X=h;
        h = self.cnn_layer(X, layer_i=2,bordor_mode="full");X=h  #*difference
        h = self.relu_layer(X);X=h;
        h = self.maxpooling_layer(X);X=h;
        h = self.dropout_layer(X, .25);X=h; #*difference
        h = self.flatten_layer(X, layer_i=7);X=h;
        h = self.dense_layer(X, fully, layer_i = 10);X=h; #*difference 
        h = self.softmax_layer2D(X);X=h;
        
        #choose max 
        max_i = self.classify(X)
        return max_i[0] #return max_i[o]
    
    
    ''' Sub Process to Deep in Layer '''
    
    ''' Cnn Layer '''
    def cnn_layer(self, X, layer_i=0, boder_mode="full"):
        features = self.layers[layer_i]["param_0"]
        bias = self.layers[layer_i]["param_1"]
        
        #define filter maxtrick 
        path_dim = features[0].shape[-1]
        #how many features do we have?
        nb_features = features[0].shape[0]
        #size image 
        image_dim = X.shape[2]
        #RGB values 
        image_channals = X.shape[1]
        #how many images do we have ?
        nb_images = X.shape[0]
        
        #border mode is help to get output full image to filter or small image than to filter 
        if boder_mode == "full":
            conv_dim = image_dim + path_dim-1
        elif boder_mode == "valid":
            conv_dim = image_dim - path_dim+1
        
        #initalize our feature matrix 
        convolved_features = np.zeros((nb_images, nb_features, conv_dim,conv_dim))  #2 conv_dim because have 2 cnn_layer
        
        #iterate through each image that we have
        for image_i in range(nb_images):
             #for each featues 
            for feature_i in range(features):
                #init a convolved images as empty 
                convolved_image = np.zeros((conv_dim, conv_dim))
                #then for each channell(rgb)
                for channel in range(image_channals):
                    #let extract a channel from our feature map 
                    feature = features[feature_i, channel,:,:]
                    #the define a channel specific part of out image 
                    image = X[image_i, channel, :,:]
                    #perform convolution on our image, using a given feature fillter 
                    convolved_image += self.convolved2d(image, feature, border_mode);
                #add bias to our convoved image 
                convolved_image = convolved_image+bias[feature_i]
                #add it to our list of convolved features (learnings)
                convolved_features[image_i, feature_i,:,:] = convolved_image 
        return convolved_features
    
    ''' Reru Layer'''
    def relu_layer(x):
        #turn all negative values in a matrix into zeros
        z = np.zeros_like(x)
        return np.where(x>z,x,z)
    
    ''' Pooling Layer ''' 
    def maxpooling_layer(self, convolved_features):
        #given our learned features and images
        nb_features = convolved_features.shape[0]
        nb_images = convolved_features.shape[1]
        conv_dim = convolved_features.shape[2] #full or vaild 
        res_dim = int(conv_dim / self.pool_size)       #assumed square shape
        
        #initialize our more dense feature list as empty
        pooled_features = np.zeros((nb_features, nb_images, res_dim, res_dim)) #same feature maxtrix
        
        #for each image
        for image_i in range(nb_images):
            #and each feature map
            for feature_i in range(nb_features):
                #begin by the row
                for pool_row in range(res_dim):
                    #define start and end points
                    row_start = pool_row * self.pool_size
                    row_end   = row_start + self.pool_size
                    #for each column (so its a 2D iteration)
                    for pool_col in range(res_dim):
                        #define start and end points
                        col_start = pool_col * self.pool_size
                        col_end   = col_start + self.pool_size
                        #define a patch given our defined starting ending points
                        patch = convolved_features[feature_i, image_i, row_start : row_end,col_start : col_end]
                        #then take the max value from that patch
                        #store it. this is our new learned feature/filter
                        pooled_features[feature_i, image_i, pool_row, pool_col] = np.max(patch)
                        return pooled_features
                    
    ''' Dropout Layer'''
    def dropout_layer(X, p):  #i aussume 0.25 to p 
        retain_prob = 1. - p
        X *= retain_prob
        return X
    ''' Flatten Layer '''
        #tensor transformation, less dimensions
        #tranlate -> แปลงมิติ เมททริกซ์ ให้มีขนาดน้อยลง
    def flatten_layer(X):
        flatX = np.zeros((X.shape[0],np.prod(X.shape[1:])))
        for i in range(X.shape[0]):
            flatX[i,:] = X[i].flatten(order='C')
        return flatX
    
    ''' Dense Layer '''
    #In a dense layer, every node in the layer is connected to every node in the preceding layer.
        #tranlate -> ในชั้นหนาแน่นทุกโหนดในชั้นเชื่อมต่อกับทุกโหนดในชั้นก่อนหน้า
    def dense_layer(self, X, layer_i=0):
        #so we'll initialize our weight and bias for this layer
        W = self.layers[layer_i]["param_0"]
        b = self.layers[layer_i]["param_1"]
        #and multiply it by our input (dot product)
        output = np.dot(X, W) + b
        return output
    
    ''' Soft max 2D'''
    def softmax_layer2D(w):
        #this function will calculate the probabilities of each
        #target class over all possible target classes. 
        maxes = np.amax(w, axis=1)
        maxes = maxes.reshape(maxes.shape[0], 1)
        e = np.exp(w - maxes)
        dist = e / np.sum(e, axis=1, keepdims=True)
        return dist
    
    ''' Classifier'''
        #get the largest probabililty value from the list
    def classify(X):
        return X.argmax(axis=-1)

    
    @staticmethod
    #so what does the convolution operation look like?, given an image and a feature map (filter)
    def convolve2d(image, feature, border_mode="full"):
        #we'll define the tensor dimensions of the image and the feature
        image_dim = np.array(image.shape)
        feature_dim = np.array(feature.shape)
        #as well as a target dimension
        target_dim = image_dim + feature_dim - 1
        
        #then we'll perform a fast fourier transform on both the input and the filter
        #performing a convolution can be written as a for loop but for many convolutions
        #this approach is too comp. expensive/slow. it can be performed orders of magnitude
        #faster using a fast fourier transform. 
        fft_result = np.fft.fft2(image, target_dim) * np.fft.fft2(feature, target_dim)
        #and set the result to our target 
        target = np.fft.ifft2(fft_result).real
        
        if border_mode == "valid":
            # To compute a valid shape, either np.all(x_shape >= y_shape) or
            # np.all(y_shape >= x_shape).
            #decide a target dimension to convolve around
            valid_dim = image_dim - feature_dim + 1
            if np.any(valid_dim < 1):
                valid_dim = feature_dim - image_dim + 1
                start_i = (target_dim - valid_dim) // 2
                end_i = start_i + valid_dim
                target = target[start_i[0]:end_i[0], start_i[1]:end_i[1]]
                return target
        

In [64]:
'''OCR Path'''
import pickle #saving and loading our serialized model 
import numpy as np #matrix math
from app.model.preprocessor import Preprocessor as img_prep #image preprocessing

#class for loading our saved model and classifying new images
class LiteOCR:
    def __init__(self, fn="alpha_weights.pkl", pool_size=2):
        #load the weights from the pickle file and the meta data
        [weights, meta] = pickle.load(open(fn, 'rb'), encoding='latin1') #currently, this class MUST be initialized from a pickle file
        #list to store labels
        self.vocab = meta["vocab"]
        
#how many rows and columns in an image
		self.img_rows = meta["img_side"] ; self.img_cols = meta["img_side"]
        
        #load our CNN
		self.CNN = LiteCNN()
        #with our saved weights
		self.CNN.load_weights(weights)
        #define the pooling layers size
		self.CNN.pool_size=int(pool_size)
    
    #classify new image
	def predict(self, image):
		print(image.shape)
        #vectorize the image into the right shape for our network
		X = np.reshape(image, (1, 1, self.img_rows, self.img_cols))
		X = X.astype("float32")
        
        #make the prediction
		predicted_i = self.CNN.predict(X)
        #return the predicted label
		return self.vocab[predicted_i]


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)